In [1]:
import time

import matplotlib.pyplot as plt
import numpy as np
import math

import cirq
import qsimcirq
import sympy as sp

import cupy as cp
from cuquantum import custatevec as cusv

from scipy.optimize import minimize

In [2]:
#Qubits on Square-Octagon Lattice w/ Lattice Parameters
qubits = [0, 1, 2, 3, 4, 5, 6, 7]
Nq = len(qubits)

xx_links = [(0, 1), (2, 3)]
yy_links = [(0, 3), (1, 2)]
zz_links = [(0, 4), (1, 5), (2, 6), (3, 7)]

Jx, Jy, Jz = -1.0/math.sqrt(2), -1.0/math.sqrt(2), -1.0
hX = hY = hZ = 0.05/math.sqrt(3)

In [3]:
#Hamiltonian Construction

#Qubit register for Hamiltonian
qreg = cirq.NamedQubit.range(Nq,prefix="q")

#Constructing Pauli Strings
XX_links_strings = []
for (i,j) in xx_links:
    XX_string = []
    for q in range(Nq):
        if q != i and q != j:
            XX_string.append(cirq.I(qreg[q]))
        else:
            XX_string.append(cirq.X(qreg[q]))
    XX_links_strings.append(XX_string)
    
YY_links_strings = []
for (i,j) in yy_links:
    YY_string = []
    for q in range(Nq):
        if q != i and q != j:
            YY_string.append(cirq.I(qreg[q]))
        else:
            YY_string.append(cirq.Y(qreg[q]))
    YY_links_strings.append(YY_string)

ZZ_links_strings = []
for (i,j) in zz_links:
    ZZ_string = []
    for q in range(Nq):
        if q != i and q != j:
            ZZ_string.append(cirq.I(qreg[q]))
        else:
            ZZ_string.append(cirq.Z(qreg[q]))
    ZZ_links_strings.append(ZZ_string)

X_qubit_strings = []
Y_qubit_strings = []
Z_qubit_strings = []
for k in qubits:
    X_string = []
    Y_string = []
    Z_string = []
    for q in range(Nq):
        if q != k:
            X_string.append(cirq.I(qreg[q]))
            Y_string.append(cirq.I(qreg[q]))
            Z_string.append(cirq.I(qreg[q]))
        else:
            X_string.append(cirq.X(qreg[q]))
            Y_string.append(cirq.Y(qreg[q]))
            Z_string.append(cirq.Z(qreg[q]))
    X_qubit_strings.append(X_string)
    Y_qubit_strings.append(Y_string)
    Z_qubit_strings.append(Z_string)

In [4]:
#Hamiltonian Construction (ctd.)

#Convert Pauli Strings into Products
XX_links_terms = []
for XX_link in XX_links_strings:
    XX_links_terms.append(cirq.PauliString(XX_link))

YY_links_terms = []
for YY_link in YY_links_strings:
    YY_links_terms.append(cirq.PauliString(YY_link))
    
ZZ_links_terms = []
for ZZ_link in ZZ_links_strings:
    ZZ_links_terms.append(cirq.PauliString(ZZ_link))

X_qubit_terms = []
for qubit_string in X_qubit_strings:
    X_qubit_terms.append(cirq.PauliString(qubit_string))

Y_qubit_terms = []
for qubit_string in Y_qubit_strings:
    Y_qubit_terms.append(cirq.PauliString(qubit_string))

Z_qubit_terms = []
for qubit_string in Z_qubit_strings:
    Z_qubit_terms.append(cirq.PauliString(qubit_string))

In [5]:
#Hamiltonian Construction (ctd.)

#Add up all the terms
Hamiltonian = 0
for XX_link in XX_links_terms:
    Hamiltonian += Jx*XX_link
for YY_link in YY_links_terms:
    Hamiltonian += Jy*YY_link
for ZZ_link in ZZ_links_terms:
    Hamiltonian += Jz*ZZ_link
for X_qubit in X_qubit_terms:
    Hamiltonian += hX*X_qubit
for Y_qubit in Y_qubit_terms:
    Hamiltonian += hY*Y_qubit
for Z_qubit in Z_qubit_terms:
    Hamiltonian += hZ*Z_qubit

In [6]:
##Hardware Efficient Ansatz 1 Construction
def HEA_1_circuit(n_layers):
    global theta_0
    global theta_1
    global theta_2
    global theta_3
    global theta_4
    global theta_5
    global theta_6
    global theta_7
    #Initialize Circuit
    HEA_1 = cirq.Circuit()
    qreg_HEA_1 = cirq.NamedQubit.range(Nq, prefix="q")
    #Declare variational parameters
    theta_0 = sp.symarray('theta_0', n_layers)
    theta_1 = sp.symarray('theta_1', n_layers)
    theta_2 = sp.symarray('theta_2', n_layers)
    theta_3 = sp.symarray('theta_3', n_layers)
    theta_4 = sp.symarray('theta_4', n_layers)
    theta_5 = sp.symarray('theta_5', n_layers)
    theta_6 = sp.symarray('theta_6', n_layers)
    theta_7 = sp.symarray('theta_7', n_layers)
    #Ansatz Construction
    for idx_layer in range(n_layers):
        HEA_1.append(cirq.ry(theta_0[idx_layer]).on(qreg_HEA_1[0]))
        HEA_1.append(cirq.ry(theta_1[idx_layer]).on(qreg_HEA_1[1]))
        HEA_1.append(cirq.ry(theta_2[idx_layer]).on(qreg_HEA_1[2]))
        HEA_1.append(cirq.ry(theta_3[idx_layer]).on(qreg_HEA_1[3]))
        HEA_1.append(cirq.ry(theta_4[idx_layer]).on(qreg_HEA_1[4]))
        HEA_1.append(cirq.ry(theta_5[idx_layer]).on(qreg_HEA_1[5]))
        HEA_1.append(cirq.ry(theta_6[idx_layer]).on(qreg_HEA_1[6]))
        HEA_1.append(cirq.ry(theta_7[idx_layer]).on(qreg_HEA_1[7]))
        HEA_1.append(cirq.CNOT(qreg_HEA_1[0],qreg_HEA_1[1]))
        HEA_1.append(cirq.CNOT(qreg_HEA_1[0],qreg_HEA_1[2]))
        HEA_1.append(cirq.CNOT(qreg_HEA_1[0],qreg_HEA_1[3]))
        HEA_1.append(cirq.CNOT(qreg_HEA_1[0],qreg_HEA_1[4]))
        HEA_1.append(cirq.CNOT(qreg_HEA_1[0],qreg_HEA_1[5]))
        HEA_1.append(cirq.CNOT(qreg_HEA_1[0],qreg_HEA_1[6]))
        HEA_1.append(cirq.CNOT(qreg_HEA_1[0],qreg_HEA_1[7]))
        HEA_1.append(cirq.CNOT(qreg_HEA_1[1],qreg_HEA_1[2]))
        HEA_1.append(cirq.CNOT(qreg_HEA_1[1],qreg_HEA_1[3]))
        HEA_1.append(cirq.CNOT(qreg_HEA_1[1],qreg_HEA_1[4]))
        HEA_1.append(cirq.CNOT(qreg_HEA_1[1],qreg_HEA_1[5]))
        HEA_1.append(cirq.CNOT(qreg_HEA_1[1],qreg_HEA_1[6]))
        HEA_1.append(cirq.CNOT(qreg_HEA_1[1],qreg_HEA_1[7]))
        HEA_1.append(cirq.CNOT(qreg_HEA_1[2],qreg_HEA_1[3]))
        HEA_1.append(cirq.CNOT(qreg_HEA_1[2],qreg_HEA_1[4]))
        HEA_1.append(cirq.CNOT(qreg_HEA_1[2],qreg_HEA_1[5]))
        HEA_1.append(cirq.CNOT(qreg_HEA_1[2],qreg_HEA_1[6]))
        HEA_1.append(cirq.CNOT(qreg_HEA_1[2],qreg_HEA_1[7]))
        HEA_1.append(cirq.CNOT(qreg_HEA_1[3],qreg_HEA_1[4]))
        HEA_1.append(cirq.CNOT(qreg_HEA_1[3],qreg_HEA_1[5]))
        HEA_1.append(cirq.CNOT(qreg_HEA_1[3],qreg_HEA_1[6]))
        HEA_1.append(cirq.CNOT(qreg_HEA_1[3],qreg_HEA_1[7]))
        HEA_1.append(cirq.CNOT(qreg_HEA_1[4],qreg_HEA_1[5]))
        HEA_1.append(cirq.CNOT(qreg_HEA_1[4],qreg_HEA_1[6]))
        HEA_1.append(cirq.CNOT(qreg_HEA_1[4],qreg_HEA_1[7]))
        HEA_1.append(cirq.CNOT(qreg_HEA_1[5],qreg_HEA_1[6]))
        HEA_1.append(cirq.CNOT(qreg_HEA_1[5],qreg_HEA_1[7]))
        HEA_1.append(cirq.CNOT(qreg_HEA_1[6],qreg_HEA_1[7]))
    return HEA_1

In [7]:
##Construct Objective Function
def objective(params, hamiltonian, qreg, simulator):
    ansatz = HEA_1_circuit(n_layers)
    params_dict = {'theta_0_0': params[0], 'theta_0_1': params[1], 'theta_0_2': params[2], 'theta_0_3': params[3], 'theta_0_4': params[4], 'theta_0_5': params[5], 'theta_0_6': params[6], 'theta_0_7': params[7], 'theta_0_8': params[8], 'theta_0_9': params[9], 'theta_1_0': params[10], 'theta_1_1': params[11], 'theta_1_2': params[12], 'theta_1_3': params[13], 'theta_1_4': params[14], 'theta_1_5': params[15], 'theta_1_6': params[16], 'theta_1_7': params[17], 'theta_1_8': params[18], 'theta_1_9': params[19], 'theta_2_0': params[20], 'theta_2_1': params[21], 'theta_2_2': params[22], 'theta_2_3': params[23], 'theta_2_4': params[24], 'theta_2_5': params[25], 'theta_2_6': params[26], 'theta_2_7': params[27], 'theta_2_8': params[28], 'theta_2_9': params[29], 'theta_3_0': params[30], 'theta_3_1': params[31], 'theta_3_2': params[32], 'theta_3_3': params[33], 'theta_3_4': params[34], 'theta_3_5': params[35], 'theta_3_6': params[36], 'theta_3_7': params[37], 'theta_3_8': params[38], 'theta_3_9': params[39], 'theta_4_0': params[40], 'theta_4_1': params[41], 'theta_4_2': params[42], 'theta_4_3': params[43], 'theta_4_4': params[44], 'theta_4_5': params[45], 'theta_4_6': params[46], 'theta_4_7': params[47], 'theta_4_8': params[48], 'theta_4_9': params[49], 'theta_5_0': params[50], 'theta_5_1': params[51], 'theta_5_2': params[52], 'theta_5_3': params[53], 'theta_5_4': params[54], 'theta_5_5': params[55], 'theta_5_6': params[56], 'theta_5_7': params[57], 'theta_5_8': params[58], 'theta_5_9': params[59], 'theta_6_0': params[60], 'theta_6_1': params[61], 'theta_6_2': params[62], 'theta_6_3': params[63], 'theta_6_4': params[64], 'theta_6_5': params[65], 'theta_6_6': params[66], 'theta_6_7': params[67], 'theta_6_8': params[68], 'theta_6_9': params[69], 'theta_7_0': params[70], 'theta_7_1': params[71], 'theta_7_2': params[72], 'theta_7_3': params[73], 'theta_7_4': params[74], 'theta_7_5': params[75], 'theta_7_6': params[76], 'theta_7_7': params[77], 'theta_7_8': params[78], 'theta_7_9': params[79]}
    expect = simulator.simulate_expectation_values(ansatz, hamiltonian, params_dict)
    return np.real(expect)

In [8]:
##Perform VQE

#Initialize the simulator
gpu_options = qsimcirq.QSimOptions(use_gpu=True, gpu_mode=1)
simulator = qsimcirq.QSimSimulator(qsim_options=gpu_options)
qreg = cirq.NamedQubit.range(Nq, prefix="q")
opt = {'disp':False,'maxiter':100000}

#Specify number of layers
n_layers = 10

#Specify number of runs
n_runs = 101

#Loop that executes VQE
start_time = time.time()
optimal_values = []
cost_function_evals_collection = []
initial_values_collection = []
optimal_parameters_collection = []
for run in range(n_runs):
    x_init = np.random.uniform(0, 2*np.pi, size=8*n_layers)
    res = minimize(objective, x0=x_init, args=(Hamiltonian, qreg, simulator), method='COBYLA', options=opt)
    optimal_values.append(res.fun)
    cost_function_evals_collection.append(res.nfev)
    initial_values_collection.append(x_init)
    optimal_parameters_collection.append(res.x)
end_time = time.time()
print(f"n_layers: {n_layers}")
print(f"Optimal Value: {min(optimal_values)}")
print(f"Cost Function Evaluations: {cost_function_evals_collection[np.argmin(optimal_values)]}")
print(f"Elapsed Time: {end_time - start_time}")
print("*" * 40)

n_layers: 10
Optimal Value: -4.691867145992684
Cost Function Evaluations: 12477
Elapsed Time: 55507.877238988876
****************************************


In [9]:
np.savetxt('Data for Diana (HEA 1 Results)/8_Qubit_10_Layer_HEA_1_Initial_Points_GPU.csv', initial_values_collection)
np.savetxt('Data for Diana (HEA 1 Results)/8_Qubit_10_Layer_HEA_1_Optimal_Values_GPU.csv', optimal_values)
np.savetxt('Data for Diana (HEA 1 Results)/8_Qubit_10_Layer_HEA_1_Optimal_Parameters_GPU.csv', optimal_parameters_collection)
np.savetxt('Data for Diana (HEA 1 Results)/8_Qubit_10_Layer_HEA_1_Cost_Function_Evaluations_GPU.csv', cost_function_evals_collection)

In [ ]:
#n_layers: 1
#Optimal Value: -4.568882238166985
#Cost Function Evaluations: 149
#Elapsed Time: 327.2972002029419
#****************************************

#n_layers: 2
#Optimal Value: -4.674339612356852
#Cost Function Evaluations: 2021
#Elapsed Time: 1045.0514822006226
#****************************************

#n_layers: 3
#Optimal Value: -4.691497699817701
#Cost Function Evaluations: 3580
#Elapsed Time: 3186.612499475479
#****************************************

#n_layers: 4
#Optimal Value: -4.691907507301012
#Cost Function Evaluations: 6533
#Elapsed Time: 5862.666325807571
#****************************************

#n_layers: 5
#Optimal Value: -4.6916899878589575
#Cost Function Evaluations: 9189
#Elapsed Time: 8917.326143741608
#****************************************

#n_layers: 10
#Optimal Value: -4.691867145992684
#Cost Function Evaluations: 12477
#Elapsed Time: 55507.877238988876
#****************************************